# 10 - FEMA Housing Registrants (IA)

Paged subset pull for TX/LA/MS/AL/FL; light clean with tract_geoid.


In [2]:
import os, time, requests, pandas as pd
from pathlib import Path

PROJECT_ROOT = Path('/Users/liamguest/LProjects/AURA/AURA')
RAW = PROJECT_ROOT / 'data' / 'raw'
INT = PROJECT_ROOT / 'data' / 'interim'
RAW.mkdir(parents=True, exist_ok=True); INT.mkdir(parents=True, exist_ok=True)

STATE_ABBR = ['TX','LA','MS','AL','FL']
states_str = "','".join(STATE_ABBR)
OPENFEMA_HOUSING_V1 = 'https://www.fema.gov/api/open/v1/IndividualAssistanceHousingRegistrantsLargeDisasters'

select_cols = ','.join([
    'id','disasterNumber','damagedStateAbbreviation','damagedZipCode','ownRent','residenceType',
    'grossIncome','specialNeeds','tsaEligible','repairAssistanceEligible','replacementAssistanceEligible',
    'personalPropertyEligible','ppfvl','censusBlockId','censusYear'
])

top = 5000
max_pages = 5
skip = 0
frames = []
for page in range(max_pages):
    params = {
        '$filter': f"damagedStateAbbreviation in ('{states_str}')",
        '$select': select_cols,
        '$format': 'json',
        '$top': top,
        '$skip': skip,
    }
    r = requests.get(OPENFEMA_HOUSING_V1, params=params, timeout=60)
    r.raise_for_status()
    items = r.json().get('IndividualAssistanceHousingRegistrantsLargeDisasters', [])
    if not items:
        break
    frames.append(pd.DataFrame(items))
    skip += top
    time.sleep(0.2)

fema = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
raw_out = RAW / 'fema_housing_subset.json'
fema.to_json(raw_out, orient='records')
print('FEMA rows:', len(fema), '->', raw_out)

def to_tract_geoid(val):
    s = str(val) if pd.notna(val) else ''
    return s[:11] if len(s) >= 11 else None

if not fema.empty:
    fema['tract_geoid'] = fema['censusBlockId'].map(to_tract_geoid)
    clean_out = INT / 'fema_housing_subset_clean.csv'
    fema.to_csv(clean_out, index=False)
    print('Wrote clean subset:', clean_out, 'rows:', len(fema))


FEMA rows: 25000 -> /Users/liamguest/LProjects/AURA/AURA/data/raw/fema_housing_subset.json
Wrote clean subset: /Users/liamguest/LProjects/AURA/AURA/data/interim/fema_housing_subset_clean.csv rows: 25000


## Aggregate FEMA to tract level (processed)

Creates tract-level features and writes `data/processed/fema_housing_by_tract.csv`.


In [4]:
import pandas as pd
from pathlib import Path

INT = Path('/Users/liamguest/LProjects/AURA/AURA/data/interim')
PROC = Path('/Users/liamguest/LProjects/AURA/AURA/data/processed'); PROC.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(INT / 'fema_housing_subset_clean.csv', dtype={'tract_geoid':'string'})
print('Loaded FEMA clean subset:', df.shape)

def rate_bool(series):
    try:
        return series.astype('boolean').mean(skipna=True)
    except Exception:
        return pd.to_numeric(series == True, errors='coerce').mean()

group_cols = ['tract_geoid','disasterNumber']
agg = df.groupby(group_cols, as_index=False).agg(
    applications=('id','count'),
    ppfvl_sum=('ppfvl','sum'),
    ppfvl_mean=('ppfvl','mean'),
    tsa_eligible_rate=('tsaEligible', rate_bool),
    owner_rate=('ownRent', lambda s: (s=='Owner').mean()),
    renter_rate=('ownRent', lambda s: (s=='Renter').mean())
)

out = PROC / 'fema_housing_by_tract_disaster.csv'
agg.to_csv(out, index=False)
print('Wrote', out, 'rows:', len(agg))

Loaded FEMA clean subset: (25000, 16)
Wrote /Users/liamguest/LProjects/AURA/AURA/data/processed/fema_housing_by_tract_disaster.csv rows: 7145
